# Mlflow keras based experiment tracking implementation in low-level and high level is explained in this tutorial 

## Mlflow is not limited to Keras it can be used with other machine learning modules such as Pytorch etc.

In [ ]:
%%capture
!pip install --quiet mlflow
!pip install --quiet pyngrok

     |████████████████████████████████| 17.0 MB 85 kB/s 
     |████████████████████████████████| 181 kB 43.6 MB/s 
     |████████████████████████████████| 209 kB 35.6 MB/s 
     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 146 kB 48.0 MB/s 
     |████████████████████████████████| 77 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 32.9 MB/s 
     |████████████████████████████████| 78 kB 2.9 MB/s 
     |████████████████████████████████| 54 kB 2.1 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
     |████████████████████████████████| 745 kB 5.0 MB/s 


##The next step would be connecting to activate ngrok engine

In [ ]:
import mlflow
get_ipython().system_raw("mlflow ui --port 5000 &")

from pyngrok import ngrok
ngrok.kill()

#Login to your ngrok account and go to "https://dashboard.ngrok.com/get-started/your-authtoken" url link
NGROK_AUTH_TOKEN = "2DSA5IG0MGfud2XGu9OFaZWfdFU_6WJbxf3X8fZZWVUCxmUVX"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr="5000", proto='http', bind_tls=True)
print(f"MLFlow tracking UI : {ngrok_tunnel.public_url}")
#click the link below 👇

MLFlow tracking UI : https://ae68-34-170-97-96.ngrok.io


###Uncomment below to stop the ngrok session

In [ ]:
#!killall ngrok

ngrok: no process found


## Stat experiment 

In [ ]:
#import numpy as np

#def traning_your_model():
#    return round(np.random.rand(),2)

In [ ]:
%%capture
from tensorflow.keras.datasets import *
from tensorflow.keras import *
from tensorflow.keras.backend import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.layers import *
from mlflow.models.signature import infer_signature

(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')
x_train = x_train/255
x_test = x_test/255
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
clear_session()

input_shape = (28,28)
nclasses = 10

model = Sequential([
                             Flatten(input_shape=input_shape),
                             Dense(units=50), 
                             layers.Activation('tanh'),
                             Dense(units=50),
                             Activation('tanh'),
                             Dense(units=nclasses), 
                             Activation('sigmoid')
                             ])
model.summary()
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
#tf_history = model.fit(x_train, y_train, batch_size=100, epochs=20, verbose=True, validation_data=(x_test, y_test))

In [ ]:
import datetime
import time

In [ ]:
model.predict(x_test)

array([[8.96224022e-01, 1.54507160e-03, 2.05487013e-04, ...,
        1.00000000e+00, 3.44716072e-01, 5.55452883e-01],
       [4.25368547e-04, 9.99986231e-01, 1.00000000e+00, ...,
        1.72641919e-07, 9.99968052e-01, 2.87622213e-04],
       [6.54823751e-08, 1.00000000e+00, 9.30206418e-01, ...,
        9.41742897e-01, 9.97311831e-01, 2.21222639e-04],
       ...,
       [5.25887742e-08, 1.60090625e-02, 6.43076419e-06, ...,
        9.88218904e-01, 8.17178249e-01, 9.94258761e-01],
       [6.91368401e-01, 9.48139859e-05, 3.31433583e-08, ...,
        1.18272169e-07, 9.99978900e-01, 8.66094232e-03],
       [8.24596584e-01, 2.95811892e-02, 8.07121992e-01, ...,
        1.02166595e-08, 7.70959735e-01, 2.42322683e-04]], dtype=float32)

In [ ]:

signature = infer_signature(x_test, model.predict(x_test))

In [ ]:
# MNIST CNN 
#def runCNN():
  # Set number of Epochs
epochs = 12



  # Using Adadelta as our optimizer
optimizer = Adadelta(1.0)
  # --- CNN Architecture End -------------------------------------------------------
  
  # MLflow Tracking
with mlflow.start_run() as run:
   # Record Start time
  start_time = time.time()
    
    # Loss function (crossentropy) and Optimizer (Adadelta)
#  model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer=optimizer,
#              metrics=['accuracy'])

  # Configure for TensorBoard visualization
  #   Reference: [Monitor progress of your Keras based neural network using Tensorboard](https://bit.ly/2C36EBJ)
  #log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  #tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
  #%load_ext tensorboard
  batch_size = 100

  # Fit our model
  #tf_history = model.fit(x_train, y_train, batch_size=100, epochs=20, verbose=True, validation_data=(x_test, y_test),callbacks=tensorboard_callback)
  tf_history = model.fit(x_train, y_train, batch_size=100, epochs=20, verbose=True, validation_data=(x_test, y_test))

  # Evaluate our model
  score = model.evaluate(x_test, y_test, verbose=0)
    
  # Record Complete Time and determine Elapsed Time
  complete_time = time.time()
  elapsed_time = complete_time - start_time

  # Log MLflow Parameters
  mlflow.log_param("mode", "single node")
  mlflow.log_param("epochs", epochs)
  mlflow.log_param("batch_size", batch_size)
  mlflow.log_param("num_classes", num_classes)
    
  # Log MLflow Metrics
  mlflow.log_metric("Test Loss", score[0])
  mlflow.log_metric("Test Accuracy", score[1])
  mlflow.log_metric("Duration", elapsed_time)
  
  signature = infer_signature(x_test, model.predict(x_test))
  # Log Model
  mlflow.keras.log_model(model, "models",signature=signature)

  # Log TF events to MLflow
  mlflow.log_artifacts(log_dir, artifact_path="tf.events")
      
    # Return RunUUID
    #return run.info
  
  # Close TF session
  #sess.close()
  
  # Complete MLflow Run
  mlflow.end_run()  

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/20
600/600 [==============================] - 3s 4ms/step - loss: 6.0933e-04 - accuracy: 0.9998 - val_loss: 0.2236 - val_accuracy: 0.9703
Epoch 2/20
600/600 [==============================] - 3s 4ms/step - loss: 1.8450e-04 - accuracy: 1.0000 - val_loss: 0.2118 - val_accuracy: 0.9722
Epoch 3/20
600/600 [==============================] - 2s 4ms/step - loss: 4.3790e-05 - accuracy: 1.0000 - val_loss: 0.2106 - val_accuracy: 0.9717
Epoch 4/20
600/600 [==============================] - 2s 4ms/step - loss: 3.1323e-05 - accuracy: 1.0000 - val_loss: 0.2106 - val_accuracy: 0.9722
Epoch 5/20
600/600 [==============================] - 2s 4ms/step - loss: 2.7172e-05 - accuracy: 1.0000 - val_loss: 0.2103 - val_accuracy: 0.9727
Epoch 6/20
600/600 [==============================] - 2s 4ms/step - loss: 2.3914e-05 - accuracy: 1.0000 - val_loss: 0.2102 - val_accuracy: 0.9726
Epoch 7/20
600/600 [==============

/usr/local/lib/python3.7/dist-packages/mlflow/keras.py:406: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input.
  "The pyfunc inference behavior of Keras models logged "
2022/08/17 00:15:30 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.8.2+zzzcolab20220719082949) contains a local version label (+zzzcolab20220719082949). MLflow logged a pip requirement for this package as 'tensorflow==2.8.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels,

##Below show you how to autolog the training process

In [ ]:
import mlflow
import mlflow.keras
# Build, compile, enable autologging, and train your model
keras_model = model
keras_model.compile(optimizer="rmsprop", loss="mse", metrics=["accuracy"])
# autolog your metrics, parameters, and model
mlflow.start_run()
mlflow.keras.autolog()
tf_history = keras_model.fit(x_train, y_train, batch_size=100, epochs=5, verbose=True, validation_data=(x_test, y_test))
mlflow.end_run()  


Epoch 1/5
600/600 [==============================] - 3s 4ms/step - loss: 9.2030e-05 - accuracy: 0.9997 - val_loss: 0.0050 - val_accuracy: 0.9719
Epoch 2/5
600/600 [==============================] - 2s 3ms/step - loss: 8.2382e-05 - accuracy: 0.9997 - val_loss: 0.0052 - val_accuracy: 0.9703
Epoch 3/5
600/600 [==============================] - 2s 4ms/step - loss: 7.7067e-05 - accuracy: 0.9998 - val_loss: 0.0049 - val_accuracy: 0.9725
Epoch 4/5
600/600 [==============================] - 2s 4ms/step - loss: 7.7561e-05 - accuracy: 0.9997 - val_loss: 0.0051 - val_accuracy: 0.9716
Epoch 5/5
600/600 [==============================] - 2s 4ms/step - loss: 7.2798e-05 - accuracy: 0.9998 - val_loss: 0.0051 - val_accuracy: 0.9716


2022/08/17 00:32:45 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.8.2+zzzcolab20220719082949) contains a local version label (+zzzcolab20220719082949). MLflow logged a pip requirement for this package as 'tensorflow==2.8.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/08/17 00:32:50 WARNING mlflow.utils.requirements_utils: Found jaxlib version (0.3.14+cuda11.cudnn805) contains a local version label (+cuda11.cudnn805). MLflow logged a pip requirement for this package as 'jaxlib==0.3.14' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/08/17 00:32:50 WARNING mlflow.utils.requirements_utils: Found tensorflow version (2.8.2+zzzcolab20220719082949) contains a local version label (+zzzcolab20220719082949). MLflo

##hyperOpt (To be integerated)

##👏	Congradulations now you know how to use Mlflow to track your experiment in Colab